In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.layers import GlobalAveragePooling2D , BatchNormalization , Dropout , Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint , LearningRateScheduler

In [2]:
data_generator = ImageDataGenerator(validation_split=0.25,
                                    rescale=1./255)

In [3]:
BATCH_SIZE = 16
#BATCH_SIZE = 8 #회사 Main PC

In [4]:
PATH = "./Dataset/AFAD-Full/Generator/"

In [5]:
train_gen = data_generator.flow_from_directory(PATH,
                                               target_size = (224, 224),
                                               batch_size = BATCH_SIZE,  
                                               subset="training",
                                               class_mode = 'binary'
                                              )

Found 124127 images belonging to 2 classes.


In [6]:
valid_gen = data_generator.flow_from_directory(PATH,
                                               target_size = (224, 224),
                                               batch_size = BATCH_SIZE,  
                                               subset="validation",
                                               class_mode = 'binary'
                                              )

Found 41374 images belonging to 2 classes.


In [7]:
log_dir = os.path.join('Logs')
CHECKPOINT_PATH = os.path.join('CheckPoints')
tb_callback = TensorBoard(log_dir=log_dir)

cp = ModelCheckpoint(filepath=CHECKPOINT_PATH, monitor='val_accuracy',
                     save_best_only = True,
                     verbose = 1)

In [8]:
initial_learning_rate = 0.01

def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * np.math.exp(-k*epoch)

In [9]:
lr_scheduler = LearningRateScheduler(lr_exp_decay, verbose=1)

In [10]:
ResNet50 = tf.keras.applications.resnet.ResNet50(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

"""
EfficientNet = tf.keras.applications.efficientnet.EfficientNetB0(
    #weights='imagenet',
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)
"""

"\nEfficientNet = tf.keras.applications.efficientnet.EfficientNetB0(\n    #weights='imagenet',\n    weights=None,\n    input_shape=(224, 224, 3),\n    include_top=False)\n"

In [11]:
#EfficientNet.trainable = False

In [12]:
DROP_OUT_RATE = 0.25

In [13]:
model= Sequential()

#model.add( EfficientNet )
model.add( ResNet50 )

model.add( GlobalAveragePooling2D() ) 
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(512, activation='relu') )
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(128, activation='relu') )
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(1, activation='sigmoid') )

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
hist = model.fit(train_gen,
                 epochs = 50,
                 validation_data=valid_gen,
                 callbacks=[cp , tb_callback , lr_scheduler],
                 verbose = 1 
)

Epoch 1/50

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.


C:\Users\Moon\anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


7758/7758 [==============================] - 909s 116ms/step - loss: 0.4102 - accuracy: 0.8096 - val_loss: 0.2898 - val_accuracy: 0.8829

Epoch 00001: val_accuracy improved from -inf to 0.88290, saving model to CheckPoints


C:\Users\Moon\anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints\assets
Epoch 2/50

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009048374180359595.
  29/7758 [..............................] - ETA: 12:41 - loss: 0.3179 - accuracy: 0.8599- ETA: 12:37 - loss: 0.3042 - accuracy: 0.8